## EMI CALCULATOR USING CLASS

In [2]:
# Importing Necessary Libraries
import pandas as pd
from datetime import datetime

class EmiCal:

    def __init__(self,principal,annual_rate,tenure_months):
        """
        Initialize the loan attributes.
    
        Parameters:
        principal (float): The principal loan amount.
        annual_rate (float): The annual interest rate (in percentage).
        tenure_months (int): The loan tenure or duration in months.
        """
        self.principal = principal
        self.annual_rate = annual_rate
        self.tenure_months = tenure_months
        self.monthly_rate = None
        self.emi = None
        self.df = pd.DataFrame()             
    
     # Function to calculate EMI
    def calculate_emi(self):
        """
    Calculate the Equated Monthly Installment (EMI) for the loan.

    This method computes the EMI based on the principal amount, annual
    interest rate, and loan tenure.

    Returns:
    float: The calculated EMI amount.
    """
        self.monthly_rate  = self.annual_rate / (12 * 100)  # monthly interest rate
        self.emi = (self.principal * self.monthly_rate * (1 + self.monthly_rate) ** self.tenure_months) / ((1 + self.monthly_rate) ** self.tenure_months - 1)
        
        return self.emi

    def populate_date_col(self):
        """
    Populate the DataFrame with date ranges for the loan tenure.

    This method creates a date range from the current date for the
    given tenure and populates the DataFrame with dates, months, and years.

    Returns:
    DataFrame: The updated DataFrame with date information.
    """
        current_year = datetime.today().year # Extracting current Year
        current_month = datetime.today().month # Extracting current Month
        current_date = datetime.today().day # Extracting current Date
        dates = pd.date_range(start=f"{current_year}-{current_month}-{current_date}",periods=self.tenure_months,freq="ME")
        self.df["Date"] = dates
        self.df["Month"] =self.df["Date"].dt.strftime('%b') ## Extracting Month
        self.df["Year"] = self.df["Date"].dt.strftime('%Y') ## Extracting Year
        
        return self.df 

    def populate_full_df(self):
        """
        Populate the DataFrame with the loan payment schedule.
    
        This method calculates the principal, interest, balance, and loan
        paid-to-date for each month of the loan tenure and updates the DataFrame.
    
        Returns:
        DataFrame: The updated DataFrame with loan payment details.
        """

        ## Create other columns for df
        #self.df["emi"] = self.emi
        self.df["Principal(A)"] = None
        self.df['Interest(B)'] = None  # Placeholder for interest paid
        self.df["Total Payment(A+B)"] = self.emi
        self.df['Balance'] = None  # Placeholder for balance
        self.df["Loan Paid To Date"] = None
    
        # Populate the schedule
        remaining_balance = self.principal
        for i in range(len(self.df)):
            interest_paid  = remaining_balance * self.monthly_rate
            principal_paid = self.emi - interest_paid
            remaining_balance-=principal_paid
            self.df.loc[i,'Principal(A)'] = round(principal_paid,2)
            self.df.loc[i,"Interest(B)"]= round(interest_paid,2)
            self.df.loc[i,"Balance"] = round(remaining_balance,2)
            self.df.loc[i,"Loan Paid To Date"] = f"{round((((self.principal - remaining_balance) /self.principal ) * 100 ),2)}%"
            
           
        return self.df

In [3]:

# Loan details
principal = 50000  # Principal loan amount in currency units
annual_rate = 12  # Annual interest rate in percentage
tenure_months = 24  # Loan tenure in months

## Create Instance of class
inst = EmiCal(principal,annual_rate,tenure_months)

##Calling respective Methods
inst.calculate_emi()
inst.populate_date_col()
df = inst.populate_full_df()


In [4]:
##Drop the 'Date' column
df.drop(columns=["Date"],inplace=True)


In [5]:
df

,Month,Year,Principal(A),Interest(B),Total Payment(A+B),Balance,Loan Paid To Date
0,Mar,2025,1853.67,500.0,2353.673611,48146.33,3.71%
1,Apr,2025,1872.21,481.46,2353.673611,46274.12,7.45%
2,May,2025,1890.93,462.74,2353.673611,44383.18,11.23%
3,Jun,2025,1909.84,443.83,2353.673611,42473.34,15.05%
4,Jul,2025,1928.94,424.73,2353.673611,40544.4,18.91%
5,Aug,2025,1948.23,405.44,2353.673611,38596.17,22.81%
6,Sep,2025,1967.71,385.96,2353.673611,36628.46,26.74%
7,Oct,2025,1987.39,366.28,2353.673611,34641.07,30.72%
8,Nov,2025,2007.26,346.41,2353.673611,32633.81,34.73%
9,Dec,2025,2027.34,326.34,2353.673611,30606.47,38.79%


In [6]:
## Yearly Balance
df.groupby(df["Year"])["Balance"].sum().reset_index()

,Year,Balance
0,2025,394927.35
1,2026,201558.93
2,2027,2330.37


In [7]:
## Showing Aggregate Yearly Matrixes
grouped_schedule = (
    df.groupby('Year', as_index=False)
    .agg({
        'Principal(A)': 'sum',  # Total Principal Paid
        'Interest(B)': 'sum',   # Total Interest Paid
        'Total Payment(A+B)': 'sum', # Total Payment (Principal + Interest)
        'Balance': 'last', # Remaining Balance at the end of the year
         'Loan Paid To Date':'last' # Total % of Amount paid
       
    })
)


In [8]:
grouped_schedule

,Year,Principal(A),Interest(B),Total Payment(A+B),Balance,Loan Paid To Date
0,2025,19393.52,4143.19,23536.736112,30606.47,38.79%
1,2026,25968.8,2275.28,28244.083334,4637.67,90.72%
2,2027,4637.67,69.68,4707.347222,0.0,100.0%
